# Debug training
May 19, 2021

In [49]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns
from functools import reduce

import socket

In [50]:
from ipywidgets import *

In [51]:
%matplotlib widget

In [52]:
base_dict={'cori':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/',
         'summit':'/autofs/nccs-svm1_home1/venkitesh/projects/cosmogan/cosmogan_pytorch/'}
facility='cori' if socket.gethostname()[:4]=='cori' else 'summit'

base_dir=base_dict[facility]

In [53]:
sys.path.append(base_dir+'/code/modules_image_analysis/')
from modules_img_analysis import *
# sys.path.append(base_dir+'/code/5_3d_cgan/1_main_code/')
# import post_analysis_pandas as post

In [54]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [55]:
dict1={'cori':{
'2d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
'3d':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3D/',
'3d_cgan':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'},
'summit':{'2d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/2d/',
          '3d':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/',
          '3d_cgan':'/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/'}}

In [69]:
# parent_dir=u.result
parent_dir=dict1[facility]['3d_cgan']
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202106*')]
dir_lst
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210620_113852_cgan_bs32_nodes8_lr0.0001-vary_fm50_…

In [72]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/20210620_63445_cgan_bs32_nodes8_lr0.0001-vary_fm50_spec0.05


## Plot Losses

In [73]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)
# df_metrics.tail(10)
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)

f_plot_metrics(df_metrics,['hist_chi'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
# f_plot_metrics(df_metrics,['lr_d','lr_g'])

In [75]:
interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [76]:
# df_metrics[(df_metrics.lr_d>=6.69e-04) ]

In [62]:
# df_metrics.plot(kind='scatter',x='step',y=['lr_d'])

In [63]:
np.unique(df_metrics.lr_d.values),np.unique(df_metrics.lr_g.values)

# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

(array([6.25e-06, 1.25e-05, 2.50e-05, 5.00e-05, 1.00e-04,      nan]),
 array([6.25e-06, 1.25e-05, 2.50e-05, 5.00e-05, 1.00e-04,      nan]))

## Calculating learn rates

In [64]:
# Nsteps=5;Lf=0.00005;Li=0.0008
# Lambda=(Lf/Li)**(1.0/Nsteps)
# print(Lambda,Lambda**2)

# lst=[10,40,60,70,80,100]
# # lst=range(1,11)
# [(Li*Lambda**(count+1),i) for count,i in enumerate(lst)]

In [65]:
# Lambda=0.25;Li=0.004
# lst=[10,40,60,70,80,100,140,180]
# # lst=range(1,11)
# [(Li*Lambda**(count+1),i) for count,i in enumerate(lst)]

## Grid plot of images

In [66]:
epochs_list=np.unique(df_metrics.epoch.values)[:-1]
def f2(epoch): 
    flist=glob.glob(result_dir+'/images/gen_img*_epoch-{0}*'.format(int(epoch)))
#     print(flist)
    steps_list=[fname.split('/')[-1].split('step-')[-1].split('.')[0] for fname in flist]
#     steps_list=df_metrics[df_metrics.epoch==epoch].step.values
    return steps_list

w2=interactive(f2,epoch=Dropdown(options=epochs_list) )
display(w2)


interactive(children=(Dropdown(description='epoch', options=(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0,…

In [68]:
result=w2.result
steps_list=result

def f3(steps):     
    for step in steps:
        
        fname=glob.glob(result_dir+'/images/gen_img_*step-{0}.npy'.format(step))[0]
#         print(fname)

        images=np.load(fname)[:,0,:,:]
#         print(images.shape)
        f_plot_grid(images[:8,:,:,0],cols=4,fig_size=(8,4))

interact(f3,steps=SelectMultiple(options=steps_list))


interactive(children=(SelectMultiple(description='steps', options=('37030', '37420', '37210', '37130', '37180'…

<function __main__.f3(steps)>